In [ ]:
import pandas as pd
import numpy as np
import sys
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import matplotlib.image as img
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import seaborn as sns
import itertools
import zipfile
import os
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

In [ ]:
zip_files = ['test1','train']
for zip_file in zip_files:
    with zipfile.ZipFile("../input/dogs-vs-cats/{}.zip".format(zip_file),"r") as z:
        z.extractall(".")
        print("{} unzipped".format(zip_file))


In [ ]:
img_folder = "../working/train"
file_name = os.listdir(img_folder)
width = 150
height = 150

In [ ]:
file_name[0:5]

In [ ]:
filenames = os.listdir("../working/train")
filepath=[]
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append("1")
    else:
        categories.append("0")
    filepath.append(os.path.join("../working/train", filename))
df = pd.DataFrame({
    'filename': filepath,
    'category': categories
})

In [ ]:
df.head()

In [ ]:
df['category'].value_counts()

In [ ]:
sns.countplot(data=df, x=df['category'])

In [ ]:
root_dir = "./train/"
def show_image(class_name, examples=2):
    labels_df=df
#     train_images_path=os.path.join(root_dir)
#     print(train_images_path)
    image_list = labels_df[labels_df['category'] == class_name]['filename'].sample(frac=1)[:examples].to_list()
    plt.figure(figsize=(20,10))
    for i, img in enumerate(image_list):
        full_path = os.path.join(img)
        img = Image.open(full_path)
        plt.subplot(1 ,examples, i%examples +1)
        plt.axis('off')
        plt.imshow(img)
        plt.title(class_name)

In [ ]:
show_image(class_name="0", examples=4)

In [ ]:
show_image(class_name="1", examples=4)

In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=seed)

In [ ]:
train.head()

In [ ]:
train_datagen = ImageDataGenerator(
rotation_range=30,
rescale=1./255,
shear_range=0.1,
zoom_range=0.4,
horizontal_flip=True,
width_shift_range=0.1,
height_shift_range=0.1)

train_datagenerator = train_datagen.flow_from_dataframe(dataframe=train,
                                                      x_col='filename',
                                                      y_col='category',
                                                      target_size=(width,height),
                                                      class_mode='binary',
                                                       batch_size=150)

In [ ]:
test_datagenerator = train_datagen.flow_from_dataframe(dataframe=test,
                                                      x_col='filename',
                                                      y_col='category',
                                                      target_size=(width,height),
                                                      class_mode='binary',
                                                       batch_size=150)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), input_shape=(width,height,3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image
plot_model(model, to_file='convent.png', show_shapes=True, show_layer_names=True)
Image(filename='convent.png')

In [ ]:
!pip install visualkeras
import visualkeras
visualkeras.layered_view(model, legend=True)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
History = model.fit(train_datagenerator,
                   epochs=10,
                   validation_data=test_datagenerator,
                   validation_steps=test.shape[0]/150,
                   steps_per_epoch=train.shape[0]/150)

In [ ]:
acc = History.history['accuracy']
val_acc = History.history['val_accuracy']
loss = History.history['loss']
val_loss = History.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'go', label='Training Loss')
plt.plot(epochs, val_loss, 'g', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save_weights("model.h5")

In [ ]:
test_loss, test_accuracy = model.evaluate(test_datagenerator, steps=len(test_datagenerator), verbose=1)
print("Loss: %.3f" %(test_loss*100.0))
print("Accuracy: %.3f" %(test_accuracy*100.0))